<a href="https://colab.research.google.com/github/ariefanwar07/ariefanwar07.github.oi/blob/main/project1_indonesia_trade_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
import subprocess
import requests
import pandas as pd
import altair as alt

def fetch_world_bank_data(country, indicator, date_range):
    url = f"https://api.worldbank.org/v2/country/{country}/indicator/{indicator}"
    params = {"date": date_range, "format": "json"}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if len(data) > 1:
            print(f"Data for {indicator} fetched successfully!")
            return pd.DataFrame(data[1])
        else:
            print(f"No data available for {indicator}.")
            return pd.DataFrame()
    else:
        print(f"Failed to fetch data for {indicator}: {response.status_code}")
        return pd.DataFrame()

def generate_chart_and_json(data_file, output_json):
    # Load and process data
    data = pd.read_csv(data_file)
    data['Year'] = pd.to_datetime(data['Year'], format='%Y')
    data_long = data.melt(id_vars=["Year"], value_vars=["Exports (US$)", "Imports (US$)", "Trade Balance (US$)"],
                          var_name="Indicator", value_name="Value")

    # Create the chart
    chart = alt.Chart(data_long).mark_line(point=True).encode(
        x=alt.X("Year:T", title="Year"),
        y=alt.Y("Value:Q", title="Value (US$)", scale=alt.Scale(zero=False)),
        color="Indicator:N",
        tooltip=["Year:T", "Indicator:N", "Value:Q"]
    ).properties(
        title="Indonesia Trade Indicators Over Time (1990–2022)",
        width=500,
        height=300
    )

    # Save as JSON
    chart.save(output_json)
    print(f"{output_json} saved.")

def push_to_github(repo_dir, commit_message="Update World Bank data and chart"):
    os.chdir(repo_dir)
    subprocess.run(["git", "add", "."])
    subprocess.run(["git", "commit", "-m", commit_message])
    subprocess.run(["git", "push", "origin", "main"])  # Replace 'main' with your branch name
    print("Changes pushed to GitHub.")

# Main script
if __name__ == "__main__":
    indicators = {
        "Exports (US$)": "NE.EXP.GNFS.CD",
        "Imports (US$)": "NE.IMP.GNFS.CD",
        "Trade Balance (US$)": "NE.RSB.GNFS.CD"
    }

    all_data = pd.DataFrame()
    for key, indicator in indicators.items():
        df = fetch_world_bank_data("ID", indicator, "1990:2022")
        if not df.empty:
            df = df[["date", "value"]].rename(columns={"date": "Year", "value": key})
            if all_data.empty:
                all_data = df
            else:
                all_data = pd.merge(all_data, df, on="Year", how="outer")

    # Save data to CSV
    data_file = "indonesia_trade_data.csv"
    all_data.to_csv(data_file, index=False)
    print(f"Data saved to {data_file}")

    # Generate chart JSON
    output_json = "project1_indonesia_trade_data.json"
    generate_chart_and_json(data_file, output_json)

Data for NE.EXP.GNFS.CD fetched successfully!
Data for NE.IMP.GNFS.CD fetched successfully!
Data for NE.RSB.GNFS.CD fetched successfully!
Data saved to indonesia_trade_data.csv
project1_indonesia_trade_data.json saved.
